<b> UFC Odds advising </b>

This notebook follows on from part 3.  Again it is mostly a learning exercise for me. I will be following the tutorial "Train a model to predict a future event" as show in the link https://www.kaggle.com/mdabbert/tutorial-2-train-a-model-to-make-bet-predictions . The original author of this work is kaggle user mdabbart".

Once the model has been made, i will plug in data based on the insights i found analysing UFC fight datasets by myself.

In [3]:
#####Setting Up

import numpy as np
import pandas as pd
import scipy as sp

#Links to different events
colby_tyron="C:/Users/dariu/Desktop/Kaggle/UFC/Fight_Data/ufc_fn_covington_woodley.csv"
israel_paulo="C:/Users/dariu/Desktop/Kaggle/UFC/Fight_Data/upcoming-event-adesanya-costa.csv"
holm_aldana="C:/Users/dariu/Desktop/Kaggle/UFC/Fight_Data/ufc_fn_holm_aldana.csv"

#####Importing data
df = pd.read_csv("C:/Users/dariu/Desktop/Kaggle/UFC/Fight_Data/ufc-master-16-09-20.csv")
df_upcoming = pd.read_csv(holm_aldana)
####Combining df
df_combined = df_upcoming.append(df)

#### Adding an ape index
df_combined["R_Ape_Index"]=df_combined["R_Reach_cms"]/df_combined["R_Height_cms"]
df_combined["B_Ape_Index"]=df_combined["B_Reach_cms"]/df_combined["B_Height_cms"]
df_combined["AI_dif"]=df_combined["R_Ape_Index"]/df_combined["B_Ape_Index"]
df_combined["AI_dif"]=df_combined["AI_dif"].apply(lambda x: 10 if x > 10 else x)


In [4]:
### Shortcut vs tutorial to make winner a numeric value
df_combined["Winner"]=df_combined["Winner"].apply(lambda x: 1 if x == "Blue" else 0)

### Making sure winner is numeric
df_combined["Winner"] = pd.to_numeric(df_combined["Winner"], errors = "coerce")

### Making sure date is in date.time  (tutorial made a mistake here, setting second df_combined as only df)
df_combined["date"]= pd.to_datetime(df_combined["date"])

###Splitting train and test set
num_upcoming = len(df_upcoming)
df_train=df_combined[num_upcoming:]
df_test=df_combined[:num_upcoming]

### Fighters set up
fighters_test = df_test[["R_fighter","B_fighter"]]
odds_test= df_test[["R_odds","B_odds"]]


###Setting up model and classifier 
from sklearn.tree import DecisionTreeClassifier
my_model= DecisionTreeClassifier(max_depth =10)
my_features = ["win_streak_dif", "height_dif", "reach_dif","sig_str_dif", "avg_td_dif","AI_dif"]

In [5]:
#### Making dataframes with only the relevant features
df_train_prepped = df_train[my_features].copy()
df_test_prepped = df_test[my_features].copy()

####Making dummified datasets
df_train_prepped = pd.get_dummies(df_train_prepped)
df_test_prepped= pd.get_dummies(df_train_prepped)

### Aligining the dummified data sets (removed because it broke the system)
##### df_train_prepped, df_test_prepped = df_train_prepped.align(df_test_prepped, join="left", axis=1)

### Fill NA
df_test_prepped=df_test_prepped.fillna(0)

fighters_test_prepped = fighters_test[fighters_test.index.isin(df_test_prepped.index)]
odds_test_prepped = odds_test[odds_test.index.isin(df_test_prepped.index)]
label_df = df_train["Winner"]
my_model.fit(df_train_prepped, label_df)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [6]:
probs = my_model.predict_proba(df_test_prepped)
df_test_prepped

,win_streak_dif,height_dif,reach_dif,sig_str_dif,avg_td_dif,AI_dif
0,0,0.00,5.08,1.980000,0.750000,0.974359
1,2,10.16,15.24,0.380000,1.840000,0.977113
2,-1,-5.08,-15.24,0.760000,0.190000,1.058345
3,0,10.16,5.08,6.920000,-1.630000,1.029815
4,0,2.54,0.00,0.580000,0.230000,1.014286
...,...,...,...,...,...,...
4350,0,0.00,2.54,-13.666667,0.000000,0.985915
4351,-3,7.62,7.62,-18.000000,-4.666667,1.002420
4352,0,-2.54,-2.12,-4.000000,1.000000,0.997516
4353,0,10.16,7.62,-40.500000,-3.500000,1.015310


In [7]:
probs

array([[0.58878505, 0.41121495],
       [0.4180791 , 0.5819209 ],
       [0.52439024, 0.47560976],
       ...,
       [0.57043478, 0.42956522],
       [0.85365854, 0.14634146],
       [0.52439024, 0.47560976]])

In [8]:
###Creating arrays of fights and odds
fighters_array= fighters_test_prepped.to_numpy()
odds_array = odds_test_prepped.to_numpy()
fighters_array
odds_array


array([[-125,  103],
       [-265,  205],
       [-150,  122],
       [-480,  342],
       [ 245, -335],
       [ 111, -137],
       [-455,  325],
       [-125,  105],
       [-150,  120],
       [-335,  246],
       [-109, -114]], dtype=int64)

In [9]:
###Putting together fighs, odds and calculated odds
prob_list = np.array(list(zip(fighters_array, odds_array, probs)))
prob_list


array([[['Holly Holm', 'Irene Aldana'],
        [-125, 103],
        [0.5887850467289719, 0.411214953271028]],

       [['Yorgan De Castro', 'Carlos Felipe'],
        [-265, 205],
        [0.4180790960451977, 0.5819209039548022]],

       [['Germaine de Randamie', 'Julianna Pena'],
        [-150, 122],
        [0.524390243902439, 0.47560975609756095]],

       [['Kyler Phillips', 'Cameron Else'],
        [-480, 342],
        [1.0, 0.0]],

       [['Dequan Townsend', 'Dusko Todorovic'],
        [245, -335],
        [0.4831932773109244, 0.5168067226890757]],

       [['Carlos Condit', 'Court McGee'],
        [111, -137],
        [0.5849056603773585, 0.41509433962264153]],

       [['Charles Jourdain', 'Josh Culibao'],
        [-455, 325],
        [0.5704347826086956, 0.4295652173913043]],

       [['Jordan Williams', 'Nassourdine Imavov'],
        [-125, 105],
        [0.6580310880829016, 0.34196891191709844]],

       [['Loma Lookboonmee', 'Jinh Yu Frey'],
        [-150, 120],
        [

In [10]:
### Setting up an odds calculator function
def get_bet_ev(odds, prob):
    if odds>0:
        return ((odds * prob)-(100*(1-prob)))
    else:
        return ((100/abs(odds))*100*prob - (100*(1-prob)))

In [11]:
#### 
for p in prob_list:
    red_ev = get_bet_ev(p[1][0], p[2][0])
    blue_ev = get_bet_ev(p[1][1], p[2][1])
    
    print(p[0][0], "(RED) vs ", p[0][1], "(BLUE)")
    print(p[0][0], "has a", "%.2f" % (p[2][0]*100), "% chance of winning. His odds are", p[1][0], "This gives him a single bet EV of", "%.2f" %red_ev)
    print(p[0][1], "has a", "%.2f" % (p[2][1]*100), "% chance of winning. His odds are", p[1][1], "This gives him a single bet EV of", "%.2f" %blue_ev)
    if red_ev >0:
        print("RED is a good bet")
    elif(blue_ev > 0):
        print("BLUE is a good bet")
    else:
        print ("There is NO good bet")
    print()

Holly Holm (RED) vs  Irene Aldana (BLUE)
Holly Holm has a 58.88 % chance of winning. His odds are -125 This gives him a single bet EV of 5.98
Irene Aldana has a 41.12 % chance of winning. His odds are 103 This gives him a single bet EV of -16.52
RED is a good bet

Yorgan De Castro (RED) vs  Carlos Felipe (BLUE)
Yorgan De Castro has a 41.81 % chance of winning. His odds are -265 This gives him a single bet EV of -42.42
Carlos Felipe has a 58.19 % chance of winning. His odds are 205 This gives him a single bet EV of 77.49
BLUE is a good bet

Germaine de Randamie (RED) vs  Julianna Pena (BLUE)
Germaine de Randamie has a 52.44 % chance of winning. His odds are -150 This gives him a single bet EV of -12.60
Julianna Pena has a 47.56 % chance of winning. His odds are 122 This gives him a single bet EV of 5.59
BLUE is a good bet

Kyler Phillips (RED) vs  Cameron Else (BLUE)
Kyler Phillips has a 100.00 % chance of winning. His odds are -480 This gives him a single bet EV of 20.83
Cameron Else h

In [12]:
#### Generating a results table

df_results = pd.DataFrame(columns=['Fighter', 'Odds', 'Win Chance', "EV"])
row = 0
for p in prob_list:
    new_row = {"Fighter":p[0][0],'Odds':p[1][0],'Win Chance':p[2][0],"EV":get_bet_ev(p[1][0], p[2][0])}
    df_results = df_results.append(new_row, ignore_index=True)
    new_row = {"Fighter":p[0][1],'Odds':p[1][1],'Win Chance':p[2][1],"EV":get_bet_ev(p[1][1], p[2][1])}
    df_results = df_results.append(new_row, ignore_index=True)
    row +=1
df_results["Advisable Bet"]=df_results["EV"].apply(lambda x: "Very Strong Yes" if x>50 else("Strong Yes" if x>25 else("Weak Yes" if x>0 else "No")))
df_results

,Fighter,Odds,Win Chance,EV,Advisable Bet
0,Holly Holm,-125,0.588785,5.981308,Weak Yes
1,Irene Aldana,103,0.411215,-16.523364,No
2,Yorgan De Castro,-265,0.418079,-42.415521,No
3,Carlos Felipe,205,0.581921,77.485876,Very Strong Yes
4,Germaine de Randamie,-150,0.524390,-12.601626,No
5,Julianna Pena,122,0.475610,5.585366,Weak Yes
6,Kyler Phillips,-480,1.000000,20.833333,Weak Yes
7,Cameron Else,342,0.000000,-100.000000,No
8,Dequan Townsend,245,0.483193,66.701681,Very Strong Yes
9,Dusko Todorovic,-335,0.516807,-32.892261,No
